# Descarga de datos del SNIIM (Sistema Nacional de información e Integración de Mercados)

El **SNIIM** proporciona información de precios al mayoreo del mercado agroalimentario a fin de coadyuvar a la toma de decisiones en materia de comercio, así como también brinda una atención a los usuarios con apego al marco legal aplicable, comprometidos en la mejora continua.

No se encontraron los datos disponibles en descarga directa, tampoco a través de APIs, por lo tanto se hace necesario realizar ***web scraping*** para la automatización y descarga de los datos.

<br>

Por ahora sólo se realiza la descarga para la categoría de **"Frutas y Hortalizas"**, las cuales son las de interés en este momento para la **Red de Banco de Alimentos (RedBAMx).**

Los parámetros por default para la descarga son los siguientes:
- Intervalo de fechas: **01/01/2020 al 31/12/2023**
- Precio por: **Kilogramo**
- Lista de productos de interés:
  
        ['Berenjena', 'Brócoli', 'Calabacita', 'Cebolla', 'Chile Anaheim', 'Chile California', 'Chile Bola', 'Chile Caloro',
        'Chile Caribe', 'Chile Cat', 'Chile Chilaca', 'Chile de Árbol fresco', 'Chile Dulce', 'Chile Habanero', 'Chile Húngaro',
        'Chile Mirasol', 'Chile Jalapeño', 'Chile Pimiento morrón', 'Chile Poblano', 'Chile Puya fresco', 'Chile Serrano',
        'Coliflor', 'Durazno', 'Espárrago', 'Frambuesa', 'Fresa', 'Guayaba', 'Jitomate', 'Lechuga', 'Limón', 'Mango',
        'Manzana', 'Melón', 'Naranja', 'Nopal', 'Nuez', 'Papa', 'Papaya', 'Pepino', 'Pera', 'Piña', 'Plátano', 'Sandía',
        'Tomate Verde', 'Toronja', 'Uva', 'Zarzamora']

<br>

*Basado en el scraper hecho por México Abierto:*
https://github.com/mxabierto/scraper-sniim/blob/master/sniim/precios_historicos.py


In [1]:
# Montar google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importar librerías

In [4]:
# Importar librerías

import re
import csv
from bs4 import BeautifulSoup
import urllib.request
#from urllib.error import HTTPError

import os
from datetime import datetime, timedelta
import pandas as pd
import logging

Se configura módulo logging para crear log de descargas, según documentación https://docs.python.org/es/3/howto/logging.html

In [5]:
#logger = logging.getLogger('my_logger')
logging.basicConfig(
    filename='./descarga.log',
    filemode='a',
    encoding='utf-8',
    format='%(asctime)s, %(levelname)s %(message)s',
    datefmt='%d/%m/%Y %I:%M:%S %p',
    level=logging.DEBUG,
    force=True # Resets any previous configuration
)

#logging.debug('This message should go to the log file')
#logging.info('So should this')
#logging.warning('And this, too')
#logging.error('And non-ASCII stuff, too, like Øresund and Malmö')

Se crean las carpetas para almacenar archivos temporales y de salida

In [6]:
# Creación de carpetas temporales y de salida
#print(os.getcwd())
path = "./drive/MyDrive/"
subdir1 = path + 'temp/'
subdir2 = path + 'raw_data/'

try:
  if not os.path.exists(subdir1):
    os.makedirs(subdir1)
  if not os.path.exists(subdir2):
    os.makedirs(subdir2)
except Exception as e:
        print ("Error al crear carpeta: ", e)


Esta función realiza el web scraping a partir de la URL del producto que recibe. Extrae la tabla con la información de los precios y la guarda en un archivo CSV.

In [7]:
# Función que realiza el scrapping para extraer una tabla de información y  guardarla en archivo CSV
def creaTabla(url, archivo_salida, paginacion, categoria):
    print (url, archivo_salida, paginacion, categoria)
    try:

        with urllib.request.urlopen(url+str(paginacion), timeout=500) as response, \
          open(path + 'temp/' + archivo_salida, 'wb') as out_file, \
          open(path + 'raw_data/' + archivo_salida + ".csv", 'w', newline="\n") as csvfile:

            data = response.read().decode('utf-8').encode('utf-8')
            soup = BeautifulSoup(data)


            try:
              paginas = soup.find('span', attrs={"id":"lblPaginacion"}).get_text()
              print(paginas, paginas != 'Página  1 de  1')

              total_paginas = paginas.split(' ')[-1]

              # Si el resultado excede a una página, entonces se vuelve a llamar la función calculando
              # paginación = número de registros por página * número total de páginas
              # con la finalidad de extraer el total de registros en un solo request
              if total_paginas != '1':
                  return creaTabla(url, archivo_salida, paginacion * int(total_paginas), categoria)
              out_file.write(data)


              table = soup.find('table',attrs={"id":"tblResultados"})

              spamwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

              for row in table.find_all("tr"):
                  x = [td.get_text() for td in row.find_all("td")]
                  spamwriter.writerow(x)


            except:
              print ('La página no tiene tabla, ', url, paginacion)

              #borrar archivos
              if os.path.exists(path + "raw_data/" + archivo_salida + ".csv"):
                csvfile.close()
                os.remove(path + "raw_data/" + archivo_salida + ".csv")
              if os.path.exists(path + "temp/" + archivo_salida):
                out_file.close()
                os.remove(path + "temp/" + archivo_salida)
              pass
    except Exception as e:
      print ("Error: ", url,e)



## Descarga automatizada de los datos

Se define la **lista de productos** de interés por descargar, así como también la **modalidad de los precios** (por presentación comercial o por kilogramo) y el **intervalo de fechas** deseado.

Una vez definidos estos parámetros, se procede a la descarga de los datos.

In [ ]:
logging.info('Descarga de datos SNIIM iniciada.')

# Se define lista de productos de interés
lista_productos = ['Berenjena', 'Brócoli', 'Calabacita', 'Cebolla', 'Chile Anaheim', 'Chile California', 'Chile Bola', 'Chile Caloro',
                  'Chile Caribe', 'Chile Cat', 'Chile Chilaca', 'Chile de Árbol fresco', 'Chile Dulce', 'Chile Habanero', 'Chile Húngaro',
                  'Chile Mirasol', 'Chile Jalapeño', 'Chile Pimiento morrón', 'Chile Poblano', 'Chile Puya fresco', 'Chile Serrano',
                  'Coliflor', 'Durazno', 'Espárrago', 'Frambuesa', 'Fresa', 'Guayaba', 'Jitomate', 'Lechuga', 'Limón', 'Mango',
                  'Manzana', 'Melón', 'Naranja', 'Nopal', 'Nuez', 'Papa', 'Papaya', 'Pepino', 'Pera', 'Piña', 'Plátano', 'Sandía',
                  'Tomate Verde', 'Toronja', 'Uva', 'Zarzamora']

# Se define si los precios se requieren por 1: Presentación comercial ó 2: Kilogramo calculado
precios_por = 2

# Se definen las fechas de búsqueda
str_fecha_inicio = "01/01/2020"
str_fecha_final = "30/12/2023"

# Se convierten a tipo date
dt_fecha_inicio = datetime.strptime(str_fecha_inicio, '%d/%m/%Y')
dt_fecha_final = datetime.strptime(str_fecha_final, '%d/%m/%Y')

# Se define un intervalo máximo de años para hacer la descarga en un solo request o en varios
max_intervalo_años = 5

# Ligas a precios
mapa_precios_url = "http://www.economia-sniim.gob.mx/mapa.asp"
base_url = 'http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas'
frutas_hortalizas_endpoint = "/ResultadosConsultaFechaFrutasYHortalizas.aspx"

# Se realiza un request para obtener las URL de los productos y se realiza el scraping
with urllib.request.urlopen(mapa_precios_url) as response, \
  open(path + 'mapa.aspx', 'wb') as out_file:
    data = response.read()#.decode('utf-8').encode('utf-8') # a `bytes` object
    out_file.write(data)
    soup_directorio = BeautifulSoup(data)

    for anchor in soup_directorio.findAll('a', string=re.compile("Precio\sde\s(?!la|los|Granos)\w+")):
        producto_id = re.search(r"ProductoId=(\d+)", anchor['href']).group(1)
        nombre_producto = anchor.string.replace("Precio de ", "") # se elimina el prefijo

        # Si el nombre de lista contiene "/" se reemplaza por "_" para prevenir error al crear archivos con este nombre
        if "/" in nombre_producto:
            nombre_producto = nombre_producto.replace("/", "_")

        #Descomentar para descargar/debuggear un producto específico
        #if nombre_producto == "Guayaba":

        # Si el producto es de interés, se descarga
        if any(p in nombre_producto for p in lista_productos):

          # Si el periodo de tiempo es mayor a 5 años, se descarga por segmentos de 5 años para prevenir error en el servidor de datos
          if(dt_fecha_final.year - dt_fecha_inicio.year) > max_intervalo_años:
            for anio_inicio in range(dt_fecha_inicio.year, dt_fecha_final.year, max_intervalo_años):
              # Si el año es mayor que el año de la fecha final, se toma el de la fecha final
              if(anio_inicio + (max_intervalo_años-1)) > dt_fecha_final.year:
                anio_final = dt_fecha_final.year
              else:
                anio_final = anio_inicio + (max_intervalo_años-1)

              url = base_url + frutas_hortalizas_endpoint + f"?ProductoId={producto_id}&fechaInicio=01/01/{anio_inicio}&fechaFinal=31/12/{anio_final}&PreciosPorId={precios_por}&RegistrosPorPagina="
              #print(url)

              if(url.find("http:") >= 0):
                creaTabla(url, nombre_producto.replace(" ","_") + "_" + str(anio_inicio) + "-" +str(anio_final), 1000, nombre_producto.replace(" ","_"))


          # De lo contrario, si el periodo de tiempo es menor a 5 años, se descarga en un solo segmento
          else:
              url = base_url + frutas_hortalizas_endpoint + f"?ProductoId={producto_id}&fechaInicio={str_fecha_inicio}&fechaFinal={str_fecha_final}&PreciosPorId={precios_por}&RegistrosPorPagina="
              #print(url)

              if(url.find("http:") >= 0):
                creaTabla(url, nombre_producto.replace(" ","_") + "_" + str(dt_fecha_inicio.year) + "-" + str(dt_fecha_final.year), 1000, nombre_producto.replace(" ","_"))

print("Descarga finalizada.")
logging.info('Descarga de datos SNIIM finalizada.')

In [40]:
info = """
        El SNIIM proporciona información de precios al mayoreo del mercado agroalimentario
        a fin de coadyuvar a la toma de decisiones en materia de comercio.

        Se realiza web scrapping utilizando las siguientes URLs:
        http://www.economia-sniim.gob.mx/mapa.asp
        http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx


        """
logging.info(info)


In [ ]:
# Se comprimen los archivos CSV en .zip
!zip -r /content/raw_data.zip /content/drive/MyDrive/raw_data

# Se descarga el archivo .zip
from google.colab import files
files.download("/content/raw_data.zip")